In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

def train_arima(pred_horizon):
    df = pd.read_csv('ISTATX_istatx1.csv', index_col='COLLECTION_TIMESTAMP')
    df.index = pd.to_datetime(df.index)
    df['COLUMN_LABEL'].unique()
    df = df.loc[df['COLUMN_LABEL'] == 'CPU Usage (per transaction)']
    df['VALUE'] = [x.replace(',', '.') for x in df['VALUE']]
    df['VALUE'] = pd.to_numeric(df['VALUE'])    
    auto_arima(df['VALUE'],seasonal=False).summary()
    train = df[:-pred_horizon].copy()
    model = ARIMA(train['VALUE'],order=(3,1,4))
    fit = model.fit()
    fit.save('arima.pkl')    
    return None    

def forecast_arima(pred_horizon):
    df = pd.read_csv('ISTATX_istatx1.csv', index_col='COLLECTION_TIMESTAMP')
    df.index = pd.to_datetime(df.index)
    df['COLUMN_LABEL'].unique()
    df = df.loc[df['COLUMN_LABEL'] == 'CPU Usage (per transaction)']
    df['VALUE'] = [x.replace(',', '.') for x in df['VALUE']]
    df['VALUE'] = pd.to_numeric(df['VALUE'])   
    train = df[:-pred_horizon].copy()
    test = df[-pred_horizon:].copy()
    start = len(train)
    end = len(train)+len(test)-1
    model = ARIMAResults.load('arima.pkl')
    predictions = model.predict(start=start, end=end, dynamic=False).rename('SARIMAX(1,1,2)(2,0,0,7) Predictions')
    print(predictions)
    return predictions.values  